# Searching for motifs

## Using MEME to find motifs

[MEME](https://meme-suite.org/meme/) must be installed to use this functionality. `find_motifs` searches for motifs upstream of all genes in an iModulon. The `gene_table` must contain the columns `accession` and `operon` for this function to work (see `notebooks/gene_annotation.ipynb`).

`find_motifs` supports many of the command-line options for MEME:

* `outdir`: Directory for output files
* `palindrome`: If True, limit search to palindromic motifs (default: False)
* `nmotifs`: Number of motifs to search for (default: 5)
* `upstream`: Number of basepairs upstream from first gene in operon to include in motif search (default: 500)
* `downstream`: Number of basepairs upstream from first gene in operon to include in motif search (default: 100)
* `verbose`: Show steps in verbose output (default: True)
* `force`: Force execution of MEME even if output already exists (default: False)
* `evt`: E-value threshold (default: 0.001)
* `cores` Number of cores to use (default: 8)
* `minw`: Minimum motif width in basepairs (default: 6)
* `maxw`: Maximum motif width in basepairs (default: 40)
* `minsites`: Minimum number of sites required for a motif. Default is the number of operons divided by 3.

In [1]:
from pymodulon.motif import *
from pymodulon.data.example_data import load_ecoli_data, ecoli_fasta
ica_data = load_ecoli_data()

In [2]:
motifs = find_motifs(ica_data, 'ArgR', ecoli_fasta)

Finding motifs for 7 sequences
Found 1 motif across 7 sites


`find_motifs` creates a `MotifInfo` object

In [3]:
motifs

<MotifInfo with 1 motif across 7 sites>

The `MotifInfo` object contains the following properties:

* `motifs`: Information about the motifs (e.g. E-value, width, consensus sequence)

In [7]:
motifs.motifs

,e_value,sites,width,consensus,motif_frac
motif,,,,,
MEME-1,1.2e-16,7,37,TGAATWWAWATKCAMTWWWTATGMATAAWWATTCANT,1.0


* `sites`: Information about predicted binding sites (e.g. location, p-value)

In [9]:
motifs.sites

rel_position    pvalue                               site_seq  \
motif  operon                                                                 
MEME-1 argA            453  2.36e-13  AGAATAAAAATACACTAATTTCGAATAATCATGCAAA   
       argCBH          375  7.76e-14  TCAATATTCATGCAGTATTTATGAATAAAAATACACT   
       argD            132  4.31e-14  TGAAATTATAACCACAAAATATGCATAAAAAATCACT   
       argF            129  2.74e-17  TGAATTAAAATTCACTTTATATGTGTAATTATTCATT   
       argG            412  2.48e-14  TAAATGAAAACTCATTTATTTTGCATAAAAATTCAGT   
       argI            127  8.81e-17  TGAATTAAAATTCAATTTATATGGATGATTATTCATT   
       artJ            171  2.66e-13  TGAATTTATATGCAATAAACATGATTAAATAATTTAA   

                        genes         locus_tags  start_pos strand  
motif  operon                                                       
MEME-1 argA              argA              b2818    2948741      +  
       argCBH  argC,argB,argH  b3958,b3959,b3960    4154500      +  
       argD              argD              b3359    3490080      -  
       argF              argF              b0273     290205      -  
       argG              argG              b3172    3318136      +  
       argI              argI              b4254    4478211      -  
       artJ              artJ              b0860     900475      -

* `cmd`: The original command used to run MEME

In [11]:
motifs.cmd

'meme motifs/ArgR.fasta -oc motifs/ArgR -dna -mod zoops -p 8 -nmotifs 5 -evt 0.001 -minw 6 -maxw 40 -allw -minsites 2'

* `file`: Path to MEME output. This is the relative path of the file from the notebook that ran `find_motifs`

In [12]:
motifs.file

'motifs/ArgR/meme.txt'

This `MotifInfo` object is automatically stored as a dictionary in the IcaData object. It will persist after saving and re-loading the IcaData object.

In [14]:
ica_data.motif_info

{'ArgR': <MotifInfo with 1 motif across 7 sites>}

## Using TOMTOM to compare motifs against external databases

Once you have a motif from MEME, you can use [TOMTOM](https://meme-suite.org/meme/tools/tomtom) to compare your motif against external databases. The `compare_motifs` function makes this process simple.

The `MotifInfo` object generated in the `find_motifs` function contains the MEME file location, which is the primary input for `compare_motifs`.

In [15]:
compare_motifs(ica_data.motif_info['ArgR'])

,motif,target,target_id,pvalue,Evalue,qvalue,overlap,optimal_offset,orientation,database
0,MEME-1,argR2,argR2,9.891940e-38,5.846130e-35,1.144640e-34,37,2,-,dpinteract
1,MEME-1,ArgR_26-5,ArgR_26-5,7.285650e-14,4.305820e-11,4.215270e-11,26,0,-,SwissRegulon_e_coli
2,MEME-1,argR,argR,1.245330e-10,7.359880e-08,2.882040e-08,18,-19,-,dpinteract
3,MEME-1,ArgR,MX000116,3.707110e-07,2.190900e-04,7.149420e-05,14,0,-,prodoric
4,MEME-1,AhrC,MX000043,5.803230e-07,3.429710e-04,8.393960e-05,16,-20,-,prodoric


This information is saved in the `matches` attribute of the ArgR `MotifInfo`. Again, this information will be maintained if you save the file to json.

In [16]:
ica_data.motif_info['ArgR'].matches

,motif,target,target_id,pvalue,Evalue,qvalue,overlap,optimal_offset,orientation,database
0,MEME-1,argR2,argR2,9.891940e-38,5.846130e-35,1.144640e-34,37,2,-,dpinteract
1,MEME-1,ArgR_26-5,ArgR_26-5,7.285650e-14,4.305820e-11,4.215270e-11,26,0,-,SwissRegulon_e_coli
2,MEME-1,argR,argR,1.245330e-10,7.359880e-08,2.882040e-08,18,-19,-,dpinteract
3,MEME-1,ArgR,MX000116,3.707110e-07,2.190900e-04,7.149420e-05,14,0,-,prodoric
4,MEME-1,AhrC,MX000043,5.803230e-07,3.429710e-04,8.393960e-05,16,-20,-,prodoric


By default, `compare_motifs` compares your motifs to 5 prokaryotic databases. You can also use the path to a local motif file or limit the search to a single database, namely:

* `'prodoric'`
* `'collectf'`
* `'dpinteract'`
* `'regtransbase'`
* `'SwissRegulon_e_coli'`

To re-run `compare_motifs` with different arguments, set `force=True`

In [17]:
compare_motifs(ica_data.motif_info['ArgR'], motif_db = 'prodoric', force=True)

,motif,target,target_id,pvalue,Evalue,qvalue,overlap,optimal_offset,orientation,database
0,MEME-1,ArgR,MX000116,4.205030e-07,0.000085,0.000076,14,0,-,prodoric
1,MEME-1,AhrC,MX000043,5.760780e-07,0.000116,0.000076,16,-20,-,prodoric
